# Data Collection

In [1]:
from keras.preprocessing.image import ImageDataGenerator

### Configure Image datagenerator class

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

### Importing the dataset

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Dtdp_3LiNzCbJKpDbMU9bdH73EmYZmHCDYFIK6AMDxSe',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-3k3fbth6tq3ve0'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

### Unzipping the dataset

In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [6]:
filenames = os.listdir('/home/wsuser/work/Dataset/train')

In [7]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


# Model Building

In [8]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [9]:
classifier = Sequential()

In [10]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

In [11]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [12]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=25, validation_data=x_test, validation_steps=len(x_test)
)

/tmp/wsuser/ipykernel_12263/572030217.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/25
119/119 [==============================] - 3s 21ms/step - loss: 1.5045 - accuracy: 0.3990 - val_loss: 0.6629 - val_accuracy: 0.8000
Epoch 2/25
119/119 [==============================] - 2s 18ms/step - loss: 0.7516 - accuracy: 0.6953 - val_loss: 0.4649 - val_accuracy: 0.8333
Epoch 3/25
119/119 [==============================] - 2s 19ms/step - loss: 0.5348 - accuracy: 0.7727 - val_loss: 0.4749 - val_accuracy: 0.8000
Epoch 4/25
119/119 [==============================] - 2s 18ms/step - loss: 0.4497 - accuracy: 0.8283 - val_loss: 0.2565 - val_accuracy: 0.9333
Epoch 5/25
119/119 [==============================] - 2s 19ms/step - loss: 0.2582 - accuracy: 0.9074 - val_loss: 0.2656 - val_accuracy: 0.9333
Epoch 6/25
119/119 [==============================] - 2s 18ms/step - loss: 0.2498 - accuracy: 0.9024 - val_loss: 0.2415 - val_accuracy: 0.9000
Epoch 7/25
119/119 [==============================] - 2s 18ms/step - loss: 0.2208 - accuracy: 0.9125 - val_loss: 0.1306 - val_accuracy: 0.9667

### Save the model

In [14]:
classifier.save('gesture.h5')

In [15]:
!tar -zcvf gesture.tgz gesture.h5

gesture.h5


In [16]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

## Testing the model

In [17]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [18]:
model = load_model("gesture.h5")

In [19]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img("/home/wsuser/work/Dataset/test/0/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

pred

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


array([[1., 0., 0., 0., 0., 0.]], dtype=float32)

In [20]:
pred[0]

array([1., 0., 0., 0., 0., 0.], dtype=float32)

# Model on IBM

In [21]:
ls -1

Dataset/
gesture.h5
gesture.tgz
model-bw.json
model.tar.gz
my_model.tar.gz


In [22]:
!pip install watson-machine-learning-client --upgrade

In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "r98pbrRMhdBZPgp-O50x4ln1MskVPOxJyOE7AB6KC3fq"
}
client = APIClient(wml_credentials)

In [24]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [25]:
space_uid = guid_from_space_name(client, 'Gesture based system')
print("Space UID = "+space_uid)

Space UID = 736a2e7c-9bcf-45c7-87f4-9a8a016b088c


In [26]:
client.set.default_space(space_uid)

'SUCCESS'

In [27]:
client.software_specifications.list(limit = 100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [28]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid 

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [29]:
model_details = client.repository.store_model(model="gesture.tgz",meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [30]:
model_id

'378c68e4-dc92-4ead-8ec6-1d2bd915a49e'

In [31]:
client.repository.download(model_id,'model.tar.gz')

File with name: 'model.tar.gz' already exists.


WMLClientError: File with name: 'model.tar.gz' already exists.